In [ ]:
!pip install transformers datasets scikit-learn


In [ ]:
from google.colab import files
uploaded = files.upload()



Saving portfolio_chatbot_intents.json to portfolio_chatbot_intents.json


In [ ]:
import json
import pandas as pd

# Load JSON
with open("portfolio_chatbot_intents.json", "r") as f:
    data = json.load(f)

# Convert to dataframe
texts = []
labels = []
for item in data:
    intent = item["intent"]
    for ex in item["examples"]:
        texts.append(ex)
        labels.append(intent)

df = pd.DataFrame({"text": texts, "label": labels})
df.head()


,text,label
0,Tell me about yourself,about_me
1,Who are you?,about_me
2,Can you introduce yourself?,about_me
3,What's your background?,about_me
4,Give me a quick intro,about_me


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

le = LabelEncoder()
df["label_enc"] = le.fit_transform(df["label"])

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label_enc"], test_size=0.2, random_state=42
)



In [ ]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=64
    )

train_encodings = tokenize(X_train)
test_encodings = tokenize(X_test)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
import torch

class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx]
        }

train_dataset = IntentDataset(train_encodings, y_train.tolist())
test_dataset = IntentDataset(test_encodings, y_test.tolist())


In [ ]:
!pip install -U transformers


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(le.classes_)
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


KeyboardInterrupt: 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "prajjwal1/bert-mini"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(le.classes_)
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

tokenizer = BertTokenizer.from_pretrained("prajjwal1/bert-mini")
model = BertForSequenceClassification.from_pretrained(
    "prajjwal1/bert-mini",
    num_labels=len(le.classes_)
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True  # Use only if running on GPU (like Colab)
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,2.298900
20,2.345000
30,2.240100
40,2.220800
50,2.167800
60,2.118200
70,2.050000
80,1.973300
90,1.909700
100,1.889000


TrainOutput(global_step=300, training_loss=1.6545487976074218, metrics={'train_runtime': 74.2234, 'train_samples_per_second': 32.335, 'train_steps_per_second': 4.042, 'total_flos': 557783539200.0, 'train_loss': 1.6545487976074218, 'epoch': 15.0})

In [ ]:
def predict(text):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return le.inverse_transform([predicted_class])[0]


test_inputs = {
    "about_me": [
        "Give me a short intro about you.",
        "Can you tell me who you are?",
        "I'd like to know more about you."
    ],
    "education": [
        "Where did you complete your studies?",
        "What’s your education background?",
        "Where did you go to university?"
    ],
    "experience": [
        "Have you worked anywhere before?",
        "What kind of experience do you have?",
        "Where have you interned or worked?"
    ],
    "projects": [
        "Have you built anything recently?",
        "Can you share your recent work?",
        "What personal projects are you proud of?"
    ],
    "skills": [
        "What kind of tech skills do you bring?",
        "Which areas are you strongest in?",
        "What technologies are you comfortable with?"
    ],
    "tech_stack": [
        "Name the tools you often work with.",
        "What’s your usual development setup?",
        "What stack do you use most?"
    ],
    "certifications": [
        "Did you earn any online credentials?",
        "Any certifications or recognitions?",
        "Have you taken any verified courses?"
    ],
    "hobbies": [
        "What do you enjoy outside of work?",
        "Any pastimes or leisure activities?",
        "How do you unwind after coding?"
    ],
    "contact": [
        "What's the best way to message you?",
        "Where can someone connect with you?",
        "How do people usually reach out to you?"
    ],
    "resume": [
        "Can you share your professional summary?",
        "Do you have a résumé to share?",
        "Where’s your CV located?"
    ]
}

for intent, questions in test_inputs.items():
    print(f"\nExpected Intent: {intent}")
    for q in questions:
        pred = predict(q)
        print(f"Q: {q}\nPredicted: {pred}\n")



Expected Intent: about_me
Q: Give me a short intro about you.
Predicted: about_me

Q: Can you tell me who you are?
Predicted: about_me

Q: I'd like to know more about you.
Predicted: about_me


Expected Intent: education
Q: Where did you complete your studies?
Predicted: education

Q: What’s your education background?
Predicted: education

Q: Where did you go to university?
Predicted: education


Expected Intent: experience
Q: Have you worked anywhere before?
Predicted: experience

Q: What kind of experience do you have?
Predicted: experience

Q: Where have you interned or worked?
Predicted: experience


Expected Intent: projects
Q: Have you built anything recently?
Predicted: projects

Q: Can you share your recent work?
Predicted: experience

Q: What personal projects are you proud of?
Predicted: projects


Expected Intent: skills
Q: What kind of tech skills do you bring?
Predicted: skills

Q: Which areas are you strongest in?
Predicted: tech_stack

Q: What technologies are you comfo

In [ ]:
model.save_pretrained("intent_model")
tokenizer.save_pretrained("intent_model")


('intent_model/tokenizer_config.json',
 'intent_model/special_tokens_map.json',
 'intent_model/vocab.txt',
 'intent_model/added_tokens.json')

In [ ]:
import pickle

with open("intent_model/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


In [ ]:
import shutil
shutil.make_archive("intent_model", 'zip', "intent_model")


'/content/intent_model.zip'

In [ ]:
# Assuming label encoder was fitted with these classes:
# ['about_me', 'education', 'experience', 'projects', 'skills']

print("All intents:", list(le.classes_))


All intents: ['about_me', 'certifications', 'contact', 'education', 'experience', 'hobbies', 'projects', 'resume', 'skills', 'tech_stack']


In [ ]:
import os

def get_dir_size(path='.'):
    total = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total += os.path.getsize(fp)
    return total

size_bytes = get_dir_size("intent_model")
size_mb = size_bytes / (1024 * 1024)

print(f"Model folder size: {size_mb:.2f} MB")


Model folder size: 42.85 MB


In [ ]:
from google.colab import files
files.download("intent_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>